## Optimizing the Model

There are several adjustments that can be attempted to optimize a model: 

- Dropping more or fewer columns.
- Creating more bins for rare occurrences in columns.
- Increasing or decreasing the number of values for each bin.
- Adding more neurons to a hidden layer.
- Adding more hidden layers.
- Using different activation functions for the hidden layers.
- Adding or reducing the number of epochs to the training regimen.

### Attempt 1 - 

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

## Attempt 1: Dropping more or fewer columns.

In [4]:
# What fields might not matter? Hmmm...I opened the input in Excel to look at the data and discovered: 
# 1) There were very few special conciderations, ~.08% are of special concideration
# 2) Income amount, only a 1/4 of them are given, otherwise they are $0. 
# 3) Status' is a pretty unlikely indicator, also. .01% (This is 'marginal'. It isn't going to 'show anything. '  

# Drop identified 'throw-aways'
application_df = application_df.drop(columns=['SPECIAL_CONSIDERATIONS', 'INCOME_AMT', 'STATUS'])

In [5]:
# know your data: How many buckets should be created from the application type column? 
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(f"{app_type_counts},\n")

# Determine which values to replace if counts are less than 1000?
replace_app_type = list(app_type_counts[app_type_counts < 1000].index)

# Replace in dataframe
for app in replace_app_type:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app, "Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64,



T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# know your data: How many buckets should be created from the classification column? 
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts.head(20)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
Name: CLASSIFICATION, dtype: int64

In [7]:
# Determine which values to replace if counts are less than 500?
replace_class = list(classification_counts[classification_counts < 500].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls, "Other")
    
# Check to make sure binning was successful
print(application_df.CLASSIFICATION.value_counts())
print(f'\n')
print(application_df.nunique())

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64


APPLICATION_TYPE       6
AFFILIATION            6
CLASSIFICATION         7
USE_CASE               5
ORGANIZATION           4
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64


In [8]:
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,ASK_AMT,IS_SUCCESSFUL
0,Other,Independent,C1000,ProductDev,Association,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,6692,1
4,T3,Independent,C1000,Heathcare,Trust,142590,1


In [9]:
# generate our categorical variable list
# categorical preprocessing can be done easiest using Dataframe.dtypes == 'object'
application_categories = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_categories

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION']

In [10]:
# Checking the number of unique values in each column
application_df[application_categories].nunique()

APPLICATION_TYPE    6
AFFILIATION         6
CLASSIFICATION      7
USE_CASE            5
ORGANIZATION        4
dtype: int64

In [11]:
# NOTE: Scikit-learn is flexible enough to perform all of the one-hot encodings at the same time.
#       Remember, the only difference from our single variable examples is that we need to pass our 
#       categorical variable list

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_categories]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_categories)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,...,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [12]:
# Now that our categorical variables have been encoded, 
# they are ready to replace our unencoded categorical 
# variables in our dataset.

# TWO STEP REPLACE: 

# Merge one-hot encoded features 
application_df = application_df.merge(encode_df,left_index=True, right_index=True)

# Drop the original application categories
application_df = application_df.drop(columns=application_categories)
application_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [13]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(columns=["IS_SUCCESSFUL"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
print(number_input_features)

hidden_nodes_layer1 =  6
hidden_nodes_layer2 = 5
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

29
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 180       
                                                                 
 dense_1 (Dense)             (None, 5)                 35        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# checkpoint_path:
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# callback: 
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4000)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 626us/step - loss: 0.6213 - accuracy: 0.6836
Epoch 2/100
804/804 [==============================] - 0s 619us/step - loss: 0.5823 - accuracy: 0.7147
Epoch 3/100
804/804 [==============================] - 0s 606us/step - loss: 0.5773 - accuracy: 0.7176
Epoch 4/100
804/804 [==============================] - 0s 603us/step - loss: 0.5734 - accuracy: 0.7224
Epoch 5/100
761/804 [===========================>..] - ETA: 0s - loss: 0.5696 - accuracy: 0.7237
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 0s 610us/step - loss: 0.5702 - accuracy: 0.7233
Epoch 6/100
804/804 [==============================] - 0s 617us/step - loss: 0.5686 - accuracy: 0.7248
Epoch 7/100
804/804 [==============================] - 1s 662us/step - loss: 0.5674 - accuracy: 0.7247
Epoch 8/100
804/804 [==============================] - 1s 623us/step - loss: 0.5664 - accuracy: 0.7255
Epoch 9/100
804/804 [=================

804/804 [==============================] - 0s 608us/step - loss: 0.5574 - accuracy: 0.7276
Epoch 64/100
804/804 [==============================] - 0s 611us/step - loss: 0.5571 - accuracy: 0.7279
Epoch 65/100
505/804 [=================>............] - ETA: 0s - loss: 0.5595 - accuracy: 0.7259
Epoch 65: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 625us/step - loss: 0.5570 - accuracy: 0.7277
Epoch 66/100
804/804 [==============================] - 1s 638us/step - loss: 0.5574 - accuracy: 0.7276
Epoch 67/100
804/804 [==============================] - 1s 629us/step - loss: 0.5569 - accuracy: 0.7284
Epoch 68/100
804/804 [==============================] - 0s 602us/step - loss: 0.5566 - accuracy: 0.7279
Epoch 69/100
804/804 [==============================] - 0s 602us/step - loss: 0.5568 - accuracy: 0.7288
Epoch 70/100
506/804 [=================>............] - ETA: 0s - loss: 0.5583 - accuracy: 0.7271
Epoch 70: saving model to checkpoints\weights.70

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5581 - accuracy: 0.7250 - 182ms/epoch - 678us/step
Loss: 0.5581141710281372, Accuracy: 0.7250145673751831


In [16]:
# Attempt 1 results saved: 
nn.save("AlphabetSoupCharity_Optimization1.h5")


## that didn't work!
We are still at 72% accuracy.
### Attempt #2 - Do both adding more hidden layers (with dropping more or fewer columns)

In [17]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(columns=["IS_SUCCESSFUL"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
print(number_input_features)

hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 10
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

29
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 10)                300       
                                                                 
 dense_4 (Dense)             (None, 10)                110       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 421
Trainable params: 421
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# checkpoint_path:
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# callback: 
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4000)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 628us/step - loss: 0.6392 - accuracy: 0.6697
Epoch 2/100
804/804 [==============================] - 1s 637us/step - loss: 0.5787 - accuracy: 0.7176
Epoch 3/100
804/804 [==============================] - 0s 606us/step - loss: 0.5719 - accuracy: 0.7202
Epoch 4/100
804/804 [==============================] - 1s 632us/step - loss: 0.5684 - accuracy: 0.7215
Epoch 5/100
718/804 [=========================>....] - ETA: 0s - loss: 0.5663 - accuracy: 0.7237
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 654us/step - loss: 0.5662 - accuracy: 0.7232
Epoch 6/100
804/804 [==============================] - 1s 629us/step - loss: 0.5648 - accuracy: 0.7256
Epoch 7/100
804/804 [==============================] - 0s 612us/step - loss: 0.5639 - accuracy: 0.7254
Epoch 8/100
804/804 [==============================] - 1s 635us/step - loss: 0.5631 - accuracy: 0.7259
Epoch 9/100
804/804 [=================

804/804 [==============================] - 0s 597us/step - loss: 0.5568 - accuracy: 0.7285
Epoch 64/100
804/804 [==============================] - 0s 609us/step - loss: 0.5568 - accuracy: 0.7287
Epoch 65/100
515/804 [==================>...........] - ETA: 0s - loss: 0.5556 - accuracy: 0.7299
Epoch 65: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 0s 618us/step - loss: 0.5567 - accuracy: 0.7292
Epoch 66/100
804/804 [==============================] - 0s 620us/step - loss: 0.5568 - accuracy: 0.7283
Epoch 67/100
804/804 [==============================] - 1s 633us/step - loss: 0.5564 - accuracy: 0.7290
Epoch 68/100
804/804 [==============================] - 0s 590us/step - loss: 0.5570 - accuracy: 0.7295
Epoch 69/100
804/804 [==============================] - 0s 589us/step - loss: 0.5565 - accuracy: 0.7295
Epoch 70/100
514/804 [==================>...........] - ETA: 0s - loss: 0.5557 - accuracy: 0.7307
Epoch 70: saving model to checkpoints\weights.70

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5583 - accuracy: 0.7242 - 155ms/epoch - 578us/step
Loss: 0.5583376288414001, Accuracy: 0.7241982221603394


In [20]:
# Attempt 2 results saved: 
nn.save("AlphabetSoupCharity_Optimization2.h5")

## We are still at 72%
### Attempt #3 - So we are going to try using different activation functions for the hidden layers and 
### (keep "adding more hidden layers" and "dropping of columns")
- first try using "sigmoid" for hidden layers
- second try using "tanh" for hidden layers


In [21]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(columns=["IS_SUCCESSFUL"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
print(number_input_features)

hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 10
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

29
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 10)                300       
                                                                 
 dense_7 (Dense)             (None, 10)                110       
                                                                 
 dense_8 (Dense)             (None, 1)                 11        
                                                                 
Total params: 421
Trainable params: 421
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# checkpoint_path:
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# callback: 
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4000)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 614us/step - loss: 0.6069 - accuracy: 0.6952
Epoch 2/100
804/804 [==============================] - 0s 617us/step - loss: 0.5727 - accuracy: 0.7234
Epoch 3/100
804/804 [==============================] - 0s 604us/step - loss: 0.5685 - accuracy: 0.7252
Epoch 4/100
804/804 [==============================] - 0s 602us/step - loss: 0.5661 - accuracy: 0.7257
Epoch 5/100
723/804 [=========================>....] - ETA: 0s - loss: 0.5647 - accuracy: 0.7265
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 649us/step - loss: 0.5651 - accuracy: 0.7262
Epoch 6/100
804/804 [==============================] - 1s 636us/step - loss: 0.5641 - accuracy: 0.7264
Epoch 7/100
804/804 [==============================] - 0s 613us/step - loss: 0.5627 - accuracy: 0.7261
Epoch 8/100
804/804 [==============================] - 0s 614us/step - loss: 0.5618 - accuracy: 0.7268
Epoch 9/100
804/804 [=================

804/804 [==============================] - 0s 607us/step - loss: 0.5537 - accuracy: 0.7314
Epoch 64/100
804/804 [==============================] - 0s 590us/step - loss: 0.5539 - accuracy: 0.7310
Epoch 65/100
486/804 [=================>............] - ETA: 0s - loss: 0.5497 - accuracy: 0.7352
Epoch 65: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 633us/step - loss: 0.5538 - accuracy: 0.7318
Epoch 66/100
804/804 [==============================] - 1s 671us/step - loss: 0.5540 - accuracy: 0.7318
Epoch 67/100
804/804 [==============================] - 0s 594us/step - loss: 0.5538 - accuracy: 0.7318
Epoch 68/100
804/804 [==============================] - 0s 609us/step - loss: 0.5532 - accuracy: 0.7319
Epoch 69/100
804/804 [==============================] - 0s 602us/step - loss: 0.5539 - accuracy: 0.7318
Epoch 70/100
513/804 [==================>...........] - ETA: 0s - loss: 0.5491 - accuracy: 0.7369
Epoch 70: saving model to checkpoints\weights.70

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5576 - accuracy: 0.7258 - 161ms/epoch - 599us/step
Loss: 0.5576242208480835, Accuracy: 0.7258309125900269


In [24]:
# Attempt 3 results saved: 
nn.save("AlphabetSoupCharity_Optimization3.h5")

## Results using activation = "sigmoid" : 
268/268 - 0s - loss: 0.5565 - accuracy: 0.7254 - 180ms/epoch - 673us/step

Loss: 0.5565040707588196, Accuracy: 0.7253644466400146

## Results using activation = "tanh" :
268/268 - 0s - loss: 0.5570 - accuracy: 0.7263 - 155ms/epoch - 578us/step

Loss: 0.5569921135902405, Accuracy: 0.7262973785400391

Still no cookies. We are still at 72% after these attempts. 
I am guessing at this point, we might not be able to increase accuracy without using a different model. 